# Problem 3.8

In [1]:
qui import delim ./data/chapter03/Titiunik_WorkingPaper_2010, clear
set seed 1234567

## Part a

In [2]:
qui rename term2year D
qui rename bills_introduced Y
qui rename texas0_arkansas1 block

In [3]:
qui tabstat Y if block ==0, by(D) stat(mean) save	
scalar ate_texas = el(r(Stat2),1,1) - el(r(Stat1),1,1)

qui tabstat Y if block ==1, by(D) stat(mean) save	
scalar ate_ark = el(r(Stat2),1,1) - el(r(Stat1),1,1)

In [7]:
di "ate_texas ="%18.5f ate_texas 

ate_texas =         -16.74167


In [8]:
di "ate_arkansas ="%18.5f ate_ark

ate_arkansas =         -10.09477


## Part b

In [9]:
qui tabstat Y if block ==0, by(D) stat(v n) save	

scalar se_texas = sqrt(el(r(Stat2),1,1)/el(r(Stat2),2,1) + /// 
					el(r(Stat1),1,1)/el(r(Stat1),2,1))
					

qui tabstat Y if block ==1, by(D) stat(v n) save	

scalar se_arkansas = sqrt(el(r(Stat2),1,1)/el(r(Stat2),2,1) + /// 
					el(r(Stat1),1,1)/el(r(Stat1),2,1))

In [12]:
di "se_texas ="%18.6f se_texas

se_texas =          9.345871


In [13]:
di "se_arkansas ="%18.6f se_arkansas

se_arkansas =          3.395979


## Part c

In [16]:
qui tabstat Y, by(block) stat(n) save	

scalar ate_overall = el(r(Stat1),1,1)/_N*ate_texas + /// 
					 el(r(Stat2),1,1)/_N*ate_ark

di "overall ate ="%18.4f ate_overall




overall ate =          -13.2168


## Part e

In [17]:
scalar se_overall = sqrt((el(r(Stat1),1,1)/_N)^2*se_texas^2 + /// 
						 (el(r(Stat2),1,1)/_N)^2*se_arkansas^2)
						 
di "overall se ="%18.5f se_overall



overall se =           4.74478


## Part f

In [18]:
// calculate probs under block assignment
qui bysort block: egen probs=mean(D)

cap program drop ate_block	
program define ate_block, rclass
args Y D probs
tempvar ipw
gen `ipw' = .
// calculate inverse probability weight under block assignment
replace `ipw' = `D'/`probs' + (1-`D')/(1-`probs')
qui reg `Y' `D' [iw=`ipw']
return scalar ate=_b[`D']
end 

In [20]:
// ssc install ritest (to install ritest package)
ritest D r(ate), strata(block) reps(10000) nodots: ///
ate_block Y D probs

(66 missing values generated)
(66 real changes made)

      command:  ate_block Y D probs
        _pm_1:  r(ate)
  res. var(s):  D
   Resampling:  Permuting D
Clust. var(s):  __00000J
     Clusters:  66
Strata var(s):  block
       Strata:  2

------------------------------------------------------------------------------
T            |     T(obs)       c       n   p=c/n   SE(p) [95% Conf. Interval]
-------------+----------------------------------------------------------------
       _pm_1 |   -13.2168      69   10000  0.0069  0.0008  .0053725   .0087244
------------------------------------------------------------------------------
Note: Confidence interval is with respect to p=c/n.
Note: c = #{|T| >= |T(obs)|}


In [21]:
// ate
di el(r(b),1,1)

-13.216796


In [22]:
// p.value.twosided
di el(r(p),1,1)

.0069
